<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter


###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [1]:
import pandas as pd
import tweepy

In [2]:
# Leer llaves


In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
#recolectar tweets
tweets = tweepy.Cursor(api.search,
 q=['#wearamask -RT'], #palabra
 lang='en', #idioma
 tweet_mode='extended',
 include_rts=False).items(200)

In [5]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [6]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date',  'user_location', 'id', 'link']
data.head()

,tweet,date,user_location,id,link
0,@NPR Earthing is good for immune system &amp; ...,2020-11-14 03:05:45,,1327447602690834432,https://twitter.com/i/web/status/1327447602690...
1,Oven gloves are oppressive. When will you shee...,2020-11-14 03:05:42,"Edinburgh/Dundee, Scotland",1327447587100700673,https://twitter.com/i/web/status/1327447587100...
2,Amazing 😂 \n\n#mustwatch \n#ygk \n#WearAMask \...,2020-11-14 03:05:27,Kingston,1327447525071138822,https://twitter.com/i/web/status/1327447525071...
3,Please #WearAMask and continue to practice soc...,2020-11-14 03:05:16,"Phoenix, AZ",1327447481282490368,https://twitter.com/i/web/status/1327447481282...
4,It’s the only way we believe we’ll be safe and...,2020-11-14 03:05:08,"Harrisburg, PA",1327447446105051136,https://twitter.com/i/web/status/1327447446105...


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [7]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [8]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,user_location,id,link,clean,links,hashtags,mentions
0,@NPR Earthing is good for immune system &amp; ...,2020-11-14 03:05:45,,1327447602690834432,https://twitter.com/i/web/status/1327447602690...,Earthing is good for immune system &amp; help...,[https://t.co/yaJUFnkycs],"[#healthproblem,go, #healthcare, #Tips, #Socia...",[@NPR]
1,Oven gloves are oppressive. When will you shee...,2020-11-14 03:05:42,"Edinburgh/Dundee, Scotland",1327447587100700673,https://twitter.com/i/web/status/1327447587100...,Oven gloves are oppressive. When will you shee...,[],[#WearAMask],[]
2,Amazing 😂 \n\n#mustwatch \n#ygk \n#WearAMask \...,2020-11-14 03:05:27,Kingston,1327447525071138822,https://twitter.com/i/web/status/1327447525071...,Amazing 😂 \n\n \n \n \n,[https://t.co/srlnVdFiHz],"[#mustwatch, #ygk, #WearAMask, #COVID19Ontario]",[]
3,Please #WearAMask and continue to practice soc...,2020-11-14 03:05:16,"Phoenix, AZ",1327447481282490368,https://twitter.com/i/web/status/1327447481282...,Please and continue to practice social distan...,[https://t.co/5i3n0V7qux],"[#WearAMask, #COVID]",[]
4,It’s the only way we believe we’ll be safe and...,2020-11-14 03:05:08,"Harrisburg, PA",1327447446105051136,https://twitter.com/i/web/status/1327447446105...,It’s the only way we believe we’ll be safe and...,[https://t.co/0SiwT4wfSi],[#WearAMask],[]


In [9]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [10]:
data.head()

,tweet,date,user_location,id,link,clean,links,hashtags,mentions,emojis
0,@NPR Earthing is good for immune system &amp; ...,2020-11-14 03:05:45,,1327447602690834432,https://twitter.com/i/web/status/1327447602690...,Earthing is good for immune system &amp; help...,[https://t.co/yaJUFnkycs],"[#healthproblem,go, #healthcare, #Tips, #Socia...",[@NPR],"[👣, 🌍, 🙏, 👥, 🧟, 🦠, ☠️]"
1,Oven gloves are oppressive. When will you shee...,2020-11-14 03:05:42,"Edinburgh/Dundee, Scotland",1327447587100700673,https://twitter.com/i/web/status/1327447587100...,Oven gloves are oppressive. When will you shee...,[],[#WearAMask],[],[🤡]
2,Amazing 😂 \n\n#mustwatch \n#ygk \n#WearAMask \...,2020-11-14 03:05:27,Kingston,1327447525071138822,https://twitter.com/i/web/status/1327447525071...,Amazing 😂 \n\n \n \n \n,[https://t.co/srlnVdFiHz],"[#mustwatch, #ygk, #WearAMask, #COVID19Ontario]",[],[😂]
3,Please #WearAMask and continue to practice soc...,2020-11-14 03:05:16,"Phoenix, AZ",1327447481282490368,https://twitter.com/i/web/status/1327447481282...,Please and continue to practice social distan...,[https://t.co/5i3n0V7qux],"[#WearAMask, #COVID]",[],[]
4,It’s the only way we believe we’ll be safe and...,2020-11-14 03:05:08,"Harrisburg, PA",1327447446105051136,https://twitter.com/i/web/status/1327447446105...,It’s the only way we believe we’ll be safe and...,[https://t.co/0SiwT4wfSi],[#WearAMask],[],[]


In [11]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
10,😷,7
11,👇🏼,4
2,🙏,4
9,💔,4
19,🤬,3


In [12]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


In [14]:
trace = go.Bar(x=temp_emojis.emoji.values[:10],
               y=temp_emojis.cnt.values[:10])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)